<a href="https://colab.research.google.com/github/MarioPrado1148/DSWP_Aluno_Mario/blob/main/Notebooks%5CFifa_poucas_vari%C3%A1veis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas necessárias

In [80]:
import pandas as pd
import numpy as np

# Definições iniciais


In [81]:
d_configuracao = {
    'display.max_columns': 1000,
    'display.expand_frame_repr': True,
    'display.max_rows': 10,
    'display.precision': 2,
    'display.show_dimensions': True
                  }

for op, value in d_configuracao.items():
    pd.set_option(op, value)
    print(op, value)
from IPython.display import Markdown

display.max_columns 1000
display.expand_frame_repr True
display.max_rows 10
display.precision 2
display.show_dimensions True


# Carregar os dados

In [82]:
df_fifa = pd.read_csv('https://raw.githubusercontent.com/MarioPrado1148/DSWP/master/Dataframes/FIFA_algumas_features.csv', index_col = 'ID')

In [83]:
df_fifa.head()

,overall,potential,value,wage,nationality,position,age,preferred_foot
ID,,,,,,,,
158023,94,94,110500000,565000,Argentina,RF,31,Left
20801,94,94,77000000,405000,Portugal,ST,33,Right
190871,92,93,118500000,290000,Brazil,LW,26,Right
193080,91,93,72000000,260000,Spain,GK,27,Right
192985,91,92,102000000,355000,Belgium,RCM,27,Right


In [84]:
df_fifa.shape

(18207, 8)

In [85]:
# retorna para cada coluna de `frame`: nome da coluna, valor de exemplo, 
#     dtype da coluna, classe do valor de exemplo
# valor de exemplo é tomado na linha de `frame` indicada por `iloc`
def exemplo_linha(frame, iloc=0):
  df_info = pd.DataFrame(dict(valor_exemplo=frame.iloc[iloc].copy()))
  df_info['dtype_coluna'] = frame.dtypes.map(lambda x: x.name)
  df_info['classe_valor'] = df_info['valor_exemplo'].map(lambda x: x.__class__.__name__)
  df_info.index.name = 'nome_coluna'
  return df_info

In [86]:
exemplo_linha(df_fifa)

,valor_exemplo,dtype_coluna,classe_valor
nome_coluna,,,
overall,94,int64,int64
potential,94,int64,int64
value,110500000,int64,int64
wage,565000,int64,int64
nationality,Argentina,object,str
position,RF,object,str
age,31,int64,int64
preferred_foot,Left,object,str


# Verificação e tratamento dos Missing Values

In [87]:
df_fifa.isna().sum()

overall            0
potential          0
value              0
wage               0
nationality        0
position          60
age                0
preferred_foot    48
Length: 8, dtype: int64

In [88]:
# Como são poucas variáveis faltantes, optamos por eliminar os casos com essas variáveis.
df_fifa_semna = df_fifa.dropna()
df_fifa_semna.shape

(18147, 8)

# Identificação de casos com valor 0

## variável 'overall'

In [89]:
df_fifa_semna[df_fifa_semna['overall']==0].value_counts()

Series([], Length: 0, dtype: int64)

## variável 'potential'

In [90]:
df_fifa_semna[df_fifa_semna['potential']==0].value_counts()

Series([], Length: 0, dtype: int64)

## variável 'value'
Constatamos que a variável'value' tem 240 valores iguais a zero. Decidimos imputar a mediana.

In [91]:
df_fifa_semna[df_fifa_semna['value']==0].value_counts()

overall  potential  value  wage  nationality  position  age  preferred_foot
80       85         0      0     Argentina    CM        24   Right             1
         80         0      0     Sweden       LCB       33   Right             1
66       67         0      0     Slovenia     RB        26   Right             1
                                 Ecuador      RB        27   Right             1
         66         0      0     Iceland      RB        33   Right             1
                                                                              ..
70       70         0      0     Egypt        RB        33   Right             1
                                 Ecuador      RB        30   Right             1
                                 Bulgaria     RB        32   Right             1
                                 Bolivia      LM        28   Left              1
48       48         0      1000  England      GK        44   Right             1
Length: 240, dtype: int64

In [70]:
df_fifa_semna['value'].median()

675000.0

In [76]:
# Atribuição da mediana para os valores 0 de 'value'
df_fifa_semna.loc[df_fifa_semna['value']==0,'value'] = df_fifa_semna['value'].median()
df_fifa_semna['value'].median()

675000.0

## Variável overall

In [78]:
df_fifa_semna[df_fifa_semna['overall']==0].value_counts()

Series([], Length: 0, dtype: int64)

# variável wage

In [92]:
df_fifa_semna[df_fifa_semna['wage']==0].value_counts()

overall  potential  value  wage  nationality  position  age  preferred_foot
80       85         0      0     Argentina    CM        24   Right             1
66       71         0      0     Ecuador      CB        23   Right             1
         69         0      0     Peru         CM        25   Right             1
                                 Bolivia      RB        26   Right             1
         68         0      0     Egypt        GK        29   Right             1
                                                                              ..
70       70         0      0     Hungary      CDM       30   Right             1
                                 Egypt        RB        33   Right             1
                                 Ecuador      RB        30   Right             1
                                 Bulgaria     RB        32   Right             1
55       57         0      0     Finland      RB        26   Right             1
Length: 229, dtype: int64

In [95]:
df_fifa_semna.loc[df_fifa_semna['wage']==0,'wage'] = df_fifa_semna['wage'].median()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [96]:
df_fifa_semna['wage'].median()

3000.0

## Variável age

In [94]:
df_fifa_semna[df_fifa_semna['age']==0].value_counts()

Series([], Length: 0, dtype: int64)

# Tratamento das variáveis categóricas

In [59]:
df_fifa_categoricas = df_fifa_semna[['nationality','position','preferred_foot']]
exemplo_linha(df_fifa_categoricas)

,valor_exemplo,dtype_coluna,classe_valor
nome_coluna,,,
nationality,Argentina,object,str
position,RF,object,str
preferred_foot,Left,object,str


In [60]:
df_fifa_categoricas.describe()

,nationality,position,preferred_foot
count,18147,18147,18147
unique,164,27,2
top,England,ST,Right
freq,1657,2152,13938


In [61]:
# domínio das variáveis categóricas
#for col in ['nationality','position','preferred_foot']
df_fifa_categoricas['nationality'].value_counts().to_frame()

,nationality
England,1657
Germany,1195
Spain,1071
Argentina,936
France,911
...,...
New Caledonia,1
Rwanda,1
São Tomé & Príncipe,1
Liberia,1


In [62]:
df_fifa_categoricas['position'].value_counts().to_frame()

,position
ST,2152
GK,2025
CB,1778
CM,1394
LB,1322
...,...
CF,74
LAM,21
RAM,21
RF,16


In [63]:
df_fifa_categoricas['preferred_foot'].value_counts().to_frame()

,preferred_foot
Right,13938
Left,4209
